In [1]:
from requests.compat import urlparse, urlunparse, urljoin
from bs4 import BeautifulSoup
import re
import requests
from requests import Session, request, get
from urllib.request import urlopen
import os

In [2]:
header = {'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36'}


In [3]:
major_class_dict = {'001' : '상의', '002' : '아우터', '003' : '바지', '020' : '원피스', '022' : '스커트', '018' : '스니커즈', '005' : '신발', '004' : '가방', '054' : '여성 가방', '017' : '스포츠용품', '007' : '모자'}
print(major_class_dict.keys(), len(major_class_dict))
for key in major_class_dict:
    os.makedirs("/content/1st project/{}".format(key), exist_ok = True)

dict_keys(['001', '002', '003', '020', '022', '018', '005', '004', '054', '017', '007']) 11


In [4]:
major_class_dict = {'001' : '상의', '002' : '아우터', '003' : '바지', '020' : '원피스', '022' : '스커트', '018' : '스니커즈', '005' : '신발', '004' : '가방', '054' : '여성 가방', '017' : '스포츠용품', '007' : '모자'}


In [5]:
new_major_class_dict = {'major_category_code' : list(major_class_dict.keys()), 'major_category_name' : list(major_class_dict.values())}

In [6]:
!pip install pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
import pandas as pd

In [8]:
df = pd.DataFrame(new_major_class_dict)

In [9]:
df

,major_category_code,major_category_name
0,001,상의
1,002,아우터
2,003,바지
3,020,원피스
4,022,스커트
5,018,스니커즈
6,005,신발
7,004,가방
8,054,여성 가방
9,017,스포츠용품


In [10]:
import sqlite3

con = sqlite3.connect("./musinsa123.db")

In [11]:
df.to_sql('major_category_table', con)

In [12]:
con.close

<function Connection.close>

In [13]:
url_mj_list = list()
mid_class_dict = dict()

idx_mid_code = 0

for i in major_class_dict:
    url_majorclass = "https://www.musinsa.com/categories/item/" + i
    url_mj_list.append(url_majorclass)
    mid_class_dict[idx_mid_code] = i
    idx_mid_code +=1 

print(mid_class_dict)
print(url_mj_list)

{0: '001', 1: '002', 2: '003', 3: '020', 4: '022', 5: '018', 6: '005', 7: '004', 8: '054', 9: '017', 10: '007'}
['https://www.musinsa.com/categories/item/001', 'https://www.musinsa.com/categories/item/002', 'https://www.musinsa.com/categories/item/003', 'https://www.musinsa.com/categories/item/020', 'https://www.musinsa.com/categories/item/022', 'https://www.musinsa.com/categories/item/018', 'https://www.musinsa.com/categories/item/005', 'https://www.musinsa.com/categories/item/004', 'https://www.musinsa.com/categories/item/054', 'https://www.musinsa.com/categories/item/017', 'https://www.musinsa.com/categories/item/007']


In [14]:
# 중분류 코드가 무엇을 뜻하는지 알려주는 딕셔너리 생성
# 중분류 딕셔너리를 원소로 하는 리스트 생성
# 중분류 폴더를 생성


a = []
b = []
c = []

mid_class_list_bymajorclass= list()

for j in range(len(url_mj_list)):
    res = requests.get(url_mj_list[j], headers=header)
    soup = BeautifulSoup(res.text, "html.parser")
    mid_codes_pre = soup.find_all("li", attrs={"data-filter-key" : "d_cat_cd"})
    # print(mid_codes_pre)
    os.chdir("/content/1st project/{}".format(mid_class_dict[j])) # 대분류 코드 폴더 디렉토리 설정

    list_temp = list()

    for idx,mid_code in enumerate(mid_codes_pre):
        
        a.append(mid_code["data-filter-value"][:3]) 
        b.append(mid_code["data-filter-value"])
        c.append(mid_code["data-filter-text"][5:]) # (중분류코드, 중분류항목)
        
        list_temp.append((mid_code["data-filter-value"], mid_code["data-filter-text"][5:])) # (중분류코드, 중분류항목)
        
        os.makedirs("./{}".format(mid_code["data-filter-value"]), exist_ok=True) # 각 대분류 폴더마다 중분류 폴더 생성
    
# print(a)
# print(b)
# print(c)        
    print(list_temp)
    mid_class_dict_bymajorclass = dict(list_temp)

    


    mid_class_list_bymajorclass.append(mid_class_dict_bymajorclass)
    
    
    
print(mid_class_dict_bymajorclass)

[('001005', '맨투맨/스웨트셔츠'), ('001010', '긴소매 티셔츠'), ('001004', '후드 티셔츠'), ('001006', '니트/스웨터'), ('001003', '피케/카라 티셔츠'), ('001002', '셔츠/블라우스'), ('001008', '기타 상의'), ('001001', '반소매 티셔츠'), ('001011', '민소매 티셔츠')]
[('002012', '숏패딩/숏헤비 아우터'), ('002015', '기타 아우터'), ('002023', '플리스/뽀글이'), ('002006', '나일론/코치  재킷'), ('002009', '겨울 기타 코트'), ('002017', '트러커 재킷'), ('002007', '겨울 싱글 코트'), ('002001', '블루종/MA-1'), ('002024', '겨울 더블 코트'), ('002004', '스타디움 재킷'), ('002014', '사파리/헌팅 재킷'), ('002008', '환절기 코트'), ('002003', '슈트/블레이저 재킷'), ('002002', '레더/라이더스 재킷'), ('002019', '아노락 재킷'), ('002020', '카디건'), ('002025', '무스탕/퍼'), ('002013', '롱패딩/롱헤비 아우터'), ('002016', '패딩 베스트'), ('002018', '트레이닝 재킷'), ('002022', '후드 집업'), ('002021', '베스트')]
[('003007', '코튼 팬츠'), ('003002', '데님 팬츠'), ('003006', '기타 바지'), ('003004', '트레이닝/조거 팬츠'), ('003008', '슈트 팬츠/슬랙스'), ('003010', '점프 슈트/오버올'), ('003005', '레깅스'), ('003009', '숏 팬츠')]
[('020008', '맥시 원피스'), ('020007', '미디 원피스'), ('020006', '미니 원피스')]
[('022001', '미니스커트'), ('022002', 

In [15]:
con = sqlite3.connect("./musinsa123.db")

In [16]:
mid_class_dict = {'major_category_code' : a, 'mid_category_code' : b, 'mid_category_name' : c}

In [17]:
df2 = pd.DataFrame(mid_class_dict)

In [23]:
df2

,major_category_code,mid_category_code,mid_category_name
0,001,001005,맨투맨/스웨트셔츠
1,001,001010,긴소매 티셔츠
2,001,001004,후드 티셔츠
3,001,001006,니트/스웨터
4,001,001003,피케/카라 티셔츠
...,...,...,...
96,007,007004,버킷/사파리햇
97,007,007005,비니
98,007,007007,트루퍼
99,007,007002,헌팅캡/베레모


In [18]:
df2.to_sql('mid_category_table', con)

In [19]:
con.close()

major_class_dict = {'001' : '상의', '002' : '아우터', '003' : '바지', '020' : '원피스', '022' : '스커트', '018' : '스니커즈', '005' : '신발', '004' : '가방', '054' : '여성 가방', '017' : '스포츠용품', '007' : '모자'}
for i in major_class_dict:
    print(i, type(i))

In [ ]:
# b[0][:3]

In [ ]:
# 대분류 -> 중분류 코드 찾아 리스트 만들기

In [20]:
# for i in range(len(mid_class_list_bymajorclass)):
#     for j in mid_class_list_bymajorclass[i]:
#         url_midclass =  "https://goods.musinsa.com/api/goods/v1/review/style/list?" + "&goodsNo=" + j   
#         print(url_midclass)

In [21]:
# for mdcls in mid_class_list_bymajorclass:
#     for k in mdcls:
#         print(k, type(k), k[:3])

In [26]:
# 중분류별 상품코드와 견본 이미지 긁어오기
# [{'001001':[goodcode1, goodcode2,...],'001002':[]},{'002001'}]
list1 = list()
goodcode_list = list()
goodcode_mid_dict = dict()

goodlist = []

a = []
b = []
c = []
d = []

for mdcls in mid_class_list_bymajorclass: # 대분류 선택 020 022
    for k in mdcls: # 중분류 선택 
        url_midclass =  "https://www.musinsa.com/categories/item/" + k  
        # 중분류 별 상품 코드, 
        # goodcode_list = list()
        for page in range(1,10):

            url_midclasses = url_midclass + "?sort=emt_high&page=" + str(page)
            
            res = requests.get(url_midclasses,headers=header)
            soup = BeautifulSoup(res.text, "html.parser")

            images = soup.find_all("img", attrs={"class":"lazyload lazy"}) # 한 페이지, 90개 상품

            # print(images)
            for idx,image in enumerate(images): # idx : max90
                os.chdir("/content/1st project/{}/{}".format(k[:3], k)) # 경로 설정

#                     #print(image['src'])
                # print(idx, image)
                image_url = image["data-original"]
                if image_url.startswith("/"):
                    image_url = "https:" + image_url

# #         #         print(image_url)
                image_res = requests.get(image_url)
                image_res.raise_for_status()


                goodcode_url = image.find_parent()['href'] 
                goodcode = goodcode_url.split('/')[-1] # 상품코드
                goodcode_list.append(goodcode)

                 
                if page == 1:
                    # os.makedirs("./{}/sample".format(goodcode), exist_ok = True)
                    # os.chdir("./{}/sample".format(goodcode))
                    # print(os.getcwd())
                    a.append(k[:3])
                    b.append(k)
                    c.append(goodcode)
                    d.append(image_url)
                    # with open("{}_sample_from_page{}.jpg".format(goodcode, page), "wb") as f:
                    #     f.write(image_res.content)
                    #     f.close()
                elif page >= 2: 
                    # os.makedirs("./{}/sample".format(goodcode), exist_ok = True)
                    # os.chdir("./{}/sample".format(goodcode))
                    # print(os.getcwd())
                    # with open("{}_sample_from_page{}.jpg".format(goodcode, page), "wb") as f:
                    #     f.write(image_res.content)
                    #     f.close()
                    a.append(mdcls)
                    b.append(k)
                    c.append(goodcode)
                    d.append(image_url)
                else:
                    break

                if idx>=89:
                    break
        
#         goodcode_mid_dict[k] = goodcode_list
    
#     list1.append(goodcode_mid_dict)

# print(list1)
        
# print(goodcode_list)

# 중분류 코드를 밸류로 하는 딕셔너리 만들어서, 인덱스 순차적으로 증가할 때 
# 중분류 코드마다 상품코드 폴더들이 있고 여기에 리뷰사진, 텍스트파일 저장
# 상품코드 변경되면 디렉토리 바꿔야
# 중분류코드 변경되면 디렉토리 바꿔야

KeyboardInterrupt: ignored

In [91]:
# d

In [24]:
k = {'major_category' : a,'mid_category' : b,'goods_id' : c, 'goods_image' : d}
df3 = pd.DataFrame(k)
      #  , 'review_number' : []}

In [25]:
df3 

,major_category,mid_category,goods_id,goods_image
0,001,001005,947067,https://image.msscdn.net/images/goods_img/2019...
1,001,001005,1208839,https://image.msscdn.net/images/goods_img/2019...
2,001,001005,897632,https://image.msscdn.net/images/goods_img/2018...
3,001,001005,1557508,https://image.msscdn.net/images/goods_img/2020...
4,001,001005,956939,https://image.msscdn.net/images/goods_img/2019...
...,...,...,...,...
553,001,001008,2411104,https://image.msscdn.net/images/goods_img/2022...
554,001,001008,2148128,https://image.msscdn.net/images/goods_img/2021...
555,001,001008,2054439,https://image.msscdn.net/images/goods_img/2021...
556,001,001008,2660716,https://image.msscdn.net/images/goods_img/2022...


In [ ]:
con = sqlite3.connect("./musinsa123.db")

In [106]:
df3.to_sql('goods_table', con)


In [ ]:
con.close()

In [ ]:
df4 = {'goods_id' : [], 'review_id':[], 'review_n/ab'  :[], 'review_image' :[], 'review_sample_image' : [], 'review_date' : [], 'review_style' : []}

In [ ]:
con = sqlite3.connect("./musinsa11.db")
df2.to_sql('review_table', con)
con.close()

https://www.musinsa.com/categories/item/001001?sort=emt_high&page=1

https://www.musinsa.com/categories/item/001001?d_cat_cd=001001&brand=&list_kind=small&sort=discount_rate&sub_sort=&page=1&display_cnt=90&group_sale=&exclusive_yn=&sale_goods=&timesale_yn=&ex_soldout=&kids=&color=&price1=&price2=&shoeSizeOption=&tags=&campaign_id=&includeKeywords=&measure=

In [ ]:
# 스타일후기 사진 뽑기
# 무신사 관리자 댓글 달려 있으면 파일명 끝에 뭐 붙이기

for goodcode in goodcode_list:
    url_good =  "https://goods.musinsa.com/api/goods/v1/review/style/list?" + "&goodsNo=" + str(goodcode)
    for page in range(10):
        url_good = url_good + "&page=" + str(page+1)
        res = requests.get(url_good, headers=header)
        soup = BeautifulSoup(res.text, "html.parser")

        comments = soup.find_all(attrs={"class":"comments comments-input"})
        reviews = soup.find_all(attrs={"class":"review-contents"})
        # reviewimg_mother = soup.find_all('li', attrs={"class" : "review-content-photo__item"})
        for idx,review in enumerate(reviews):
            review_no = review["data-review-no"]
            # print(review_no)
            review_images = review.find_all('li', attrs={"class" : "review-content-photo__item"})
            for idx_img, review_image in enumerate(review_images):
                review_img_pre = review_image.find('img')["src"]
                if review_img_pre.startswith("/"):
                    review_img_url = "https:" + review_img_pre[:-5]
                print(review_img_url, idx_img)
                for idx,image in enumerate(images): # idx : max90
#                 os.chdir("/content/1st project/{}/{}".format(k[:3], k)) # 경로 설정


            # for idx_cmt, comment in enumerate(comments):
            #     if comment["c_idx"] == review_no:
            #         print('일치', comment["c_idx"])
            #         if comment.find("p", attrs={"class" : "recomment_txt"}) is not None:
            #             admincmt = comment.find("p", attrs={"class" : "recomment_txt"}).find_all('span', {"class":"cmt_detail"})
            #             print(admincmt)
            #             print(goodcode,page+1,review_no,idx+1,idx_img+1, 'Ab')
            #         else:
            #             print(goodcode,page+1,review_no,idx+1,idx_img+1, 'N')
        


https://image.msscdn.net/data/estimate/1883375_0/gallery_6357769800918.jpg 0
https://image.msscdn.net/data/estimate/1883375_0/gallery_6353e51480562.jpg 0
https://image.msscdn.net/data/estimate/1883375_0/gallery_6353e455478b7.jpg 0
https://image.msscdn.net/data/estimate/1883375_0/gallery_6353e38359395.jpg 0
https://image.msscdn.net/data/estimate/1883375_0/gallery_635384e300e1c.jpg 0
https://image.msscdn.net/data/estimate/1883375_0/gallery_6351f057a6b6e.jpg 0
https://image.msscdn.net/data/estimate/1883375_0/gallery_634eb9b48606c.jpg 0
https://image.msscdn.net/data/estimate/1883375_0/gallery_63468d84b0c22.jpg 0
https://image.msscdn.net/data/estimate/1883375_0/gallery_6341127ef32f3.jpg 0
https://image.msscdn.net/data/estimate/1883375_0/gallery_634112819fdc2.jpg 1
https://image.msscdn.net/data/estimate/1883375_0/gallery_633978c09909b.jpg 0
https://image.msscdn.net/data/estimate/1883375_0/gallery_633978c14fb39.jpg 1
https://image.msscdn.net/data/estimate/1883375_0/gallery_6357769800918.jpg 0

KeyboardInterrupt: ignored

In [ ]:
# 상품후기 사진 뽑기
# 무신사 관리자 댓글 달려 있으면 파일명 끝에 뭐 붙이기


for goodcode in goodcode_list:
    url_good =  "https://goods.musinsa.com/api/goods/v1/review/photo/list?" + "&goodsNo=" + str(goodcode)
    for page in range(10):
        url_good = url_good + "&page=" + str(page+1)
        res = requests.get(url_good, headers=header)
        soup = BeautifulSoup(res.text, "html.parser")

        comments = soup.find_all(attrs={"class":"comments comments-input"})
        reviews = soup.find_all(attrs={"class":"review-contents"})
        # reviewimg_mother = soup.find_all('li', attrs={"class" : "review-content-photo__item"})
        for idx,review in enumerate(reviews):
            review_no = review["data-review-no"]
            print(review_no)
            review_images = review.find_all('li', attrs={"class" : "review-content-photo__item"})
            for idx_img, review_image in enumerate(review_images):
                review_img_pre = review_image.find('img')["src"]
                if reviewimg_pre.startswith("/"):
                    review_img_url = "https:" + review_img_pre[:-5]
                print(review_img_url, idx_img)


            for idx_cmt, comment in enumerate(comments):
                if comment["c_idx"] == review_no:
                    print('일치', comment["c_idx"])
                    if comment.find("p", attrs={"class" : "recomment_txt"}) is not None:
                        admincmt = comment.find("p", attrs={"class" : "recomment_txt"}).find_all('span', {"class":"cmt_detail"})
                        print(admincmt)
                        print(goodcode,page+1,review_no,idx+1,idx_img+1, 'Ab')
                    else:
                        print(goodcode,page+1,review_no,idx+1,idx_img+1, 'N')
        

34095654
https://image.msscdn.net/data/estimate/1324169_0/gallery_635800b26f324.jpg 0
일치 34095654
1324169 1 34095654 1 1 N
34092442
https://image.msscdn.net/data/estimate/1324169_0/gallery_6357f29ed704b.jpg 0
일치 34092442
1324169 1 34092442 2 1 N
34041151
https://image.msscdn.net/data/estimate/1324169_0/gallery_63567fdb60238.jpg 0
일치 34041151
1324169 1 34041151 3 1 N
33998360
https://image.msscdn.net/data/estimate/1324169_0/gallery_6355234c35c54.jpg 0
일치 33998360
[<span class="cmt_detail" style="cursor:pointer;display:none;">안녕하세요. 패션을 사랑하는 무신사 CS입니다.<br/><br/>정성스럽게 올려주신 소중한 후기 감사합니다.<br/>다만, 현재 적립금 지급 기준이 충족되지 않아 적립금 지급이 보류된 상태입니다.<br/>번거로우시겠지만, 아래 기준을 참고하여 수정해 주시면 재검토 후 적립금 지급 도와드리겠습니다.<br/><br/>지급 보류 유형 7.전체 실루엣 식별 어려운 컷<br/><br/>▶ 지급 보류 상세 사유 : 상품의 형태가 프레임 내에 또렷하게 보이도록 직접 촬영한 사진으로 수정 부탁드립니다. (소매, 끝부분 등 부분적으로 잘리지 않아야 함)<br/><br/>TIP. 상품 사진 후기는 상품을 정밀하게 확인하실 수 있도록 만든 후기로 사진 등록 시 상품 전체 실루엣이 프레임 안에 전부 보여야 하며,<br/>신체의 일부만 등록되거나 상품 식별이 불가능한 경우 지급이 보류될 수 있습니다.<br/>특수기호(~!@) 또는 반복되는 글자나 후기와

In [ ]:
type(reviewimg_mother)

1

In [ ]:
url_good =  "https://goods.musinsa.com/api/goods/v1/review/style/list?" + "&goodsNo=" + str(goodcode_list[0]) + "&page=1"
res = requests.get(url_good, headers=header)
soup = BeautifulSoup(res.text, "html.parser")
reviewimg_mother = soup.find('li', attrs={"class" : "review-content-photo__item"})
# reviewimg_mother.find('img')["src"]

reviewimg_mother.find_parent(attrs={"class" : "review-list"}).parent.find(attrs={"class" : "comment-list"})

In [ ]:
reviewimg_mother.find_parent(attrs={"class" : "review-list"}).parent.find(attrs={"class" : "comment_recomment comment_wrap"})

In [ ]:
url_good =  "https://goods.musinsa.com/api/goods/v1/review/style/list?" + "&goodsNo=" + "2385875" + "&page=8"
res = requests.get(url_good, headers=header)
soup = BeautifulSoup(res.text, "html.parser")
reviewimg_mother = soup.find('li', attrs={"class" : "review-content-photo__item"})
reviewimg_mother.find_parent(attrs={"class" : "review-contents"})["data-review-no"]
# soup.find('div',attrs={"class" : "comment_recomment comment_wrap"})

'33208934'

In [ ]:
url_good =  "https://goods.musinsa.com/api/goods/v1/review/style/list?" + "&goodsNo=" + "2385875" + "&page=8"

res = requests.get(url_good, headers=header)
soup = BeautifulSoup(res.text, "html.parser")

comments = soup.find_all(attrs={"class":"comments comments-input"})
reviews = soup.find_all(attrs={"class":"review-contents"})
# reviewimg_mother = soup.find_all('li', attrs={"class" : "review-content-photo__item"})
for idx,review in enumerate(reviews):
    review_no = review["data-review-no"]
    print(review_no)
    review_images = review.find_all('li', attrs={"class" : "review-content-photo__item"})
    for idx_img, review_image in enumerate(review_images):
        review_img_pre = review_image.find('img')["src"]
        if reviewimg_pre.startswith("//"):
            review_img_url = "https:" + review_img_pre[:-5]
        print(review_img_url, idx_img)

        
    for idx_cmt, comment in enumerate(comments):
        if comment["c_idx"] == review_no:
            print('일치', comment["c_idx"])
            if comment.find("p", attrs={"class" : "recomment_txt"}) is not None:
                admincmt = comment.find("p", attrs={"class" : "recomment_txt"}).find_all('span', {"class":"cmt_detail"})
                print(admincmt)
                print('상품코드','페이지넘버',idx+1,idx_img+1, 'Ab')
            else:
                print('상품코드','페이지넘버',idx+1,idx_img+1, 'N')
        

#         print('상품코드','페이지넘버',idx+1,idx_img+1)
        # 파일명 : 상품코드_페이지넘버_idx_(idx_img)_N/Ab
#          if review_image.find_parent(attrs={})

33208934
https://image.msscdn.net/data/estimate/2385875_0/gallery_633daa8d16f0d.jpg 0
일치 33208934
상품코드 페이지넘버 1 1 N
33129853
https://image.msscdn.net/data/estimate/2385875_0/gallery_633b7a471e287.jpg 0
일치 33129853
상품코드 페이지넘버 2 1 N
33129436
https://image.msscdn.net/data/estimate/2385875_0/gallery_633b7464857f1.jpg 0
https://image.msscdn.net/data/estimate/2385875_0/gallery_633b74657382c.jpg 1
일치 33129436
상품코드 페이지넘버 3 2 N
33096627
https://image.msscdn.net/data/estimate/2385875_0/gallery_633a61ba269ee.jpg 0
일치 33096627
상품코드 페이지넘버 4 1 N
33057783
https://image.msscdn.net/data/estimate/2385875_0/gallery_6338f274b0070.jpg 0
일치 33057783
상품코드 페이지넘버 5 1 N
33038120
https://image.msscdn.net/data/estimate/2385875_0/gallery_633813ef05fb2.jpg 0
일치 33038120
[<span class="cmt_detail" style="cursor:pointer;display:none;">안녕하세요. 패션을 사랑하는 무신사 CS입니다.<br/><br/>정성스럽게 올려주신 소중한 후기 감사합니다.<br/>죄송하지만, 현재 작성해 주신 후기는  스타일 후기 적립금 지급 기준이 충족되지 않아 지급이 보류되었습니다.<br/>번거로우시겠지만, 아래 기준을 참고하여 수정해 주시면 재검토 후 적립금 지급 도와드리겠습니다.<br/>

In [ ]:

comments = soup.find_all(attrs={"class":"comments comments-input"})
for idx_cmt, comment in enumerate(comments):
    print(comment.find("p", attrs={"class" : "recomment_txt"}))

None
None
None
None
None
<p class="recomment_txt">
<span class="cmt_summary" cmt_no="8891894" style="cursor:pointer;display:block;">안녕하세요. 패션을 사랑하는 무신사 CS입니다.<br/><br/>정성스럽게 올려주신 소중한 후기 감사합니다.<br/>죄송하지만, 현재 작성해 주신 후기는  스타일 후기 적립금 지급 기준이 충족되지 않아...</span>
<span class="cmt_detail" style="cursor:pointer;display:none;">안녕하세요. 패션을 사랑하는 무신사 CS입니다.<br/><br/>정성스럽게 올려주신 소중한 후기 감사합니다.<br/>죄송하지만, 현재 작성해 주신 후기는  스타일 후기 적립금 지급 기준이 충족되지 않아 지급이 보류되었습니다.<br/>번거로우시겠지만, 아래 기준을 참고하여 수정해 주시면 재검토 후 적립금 지급 도와드리겠습니다.<br/><br/>▶ 스타일 후기 적립금 지급 조건<br/>- 해당 상품을 착용한 전신 정면 스타일링(어깨부터 발끝까지 모두 나오는) 사진<br/>- 착용이 어려운 상품은 해당 상품이 잘 보이도록 휴대 또는 함께 있는 전신 사진 <br/>- 20자 이상의 상품에 대한 평가(반복된 글자, 관련없는 내용은 X)<br/><br/>▶ 후기 수정 방법<br/>- 마이페이지 → 후기 작성 → 후기 내역(수정 희망하시는 후기 내용 클릭) → 수정/삭제<br/><br/>※ 수정된 후기의 검수 및 적립금 지급까지는 평일 기준 1~2일 정도 소요됩니다.<br/>※ 댓글은 관리자가 확인하지 않습니다. 별도 문의사항은 1:1문의 작성을 부탁드립니다.<br/><br/>문의하신 사항 외에 서비스 이용 관련 내용은 FAQ를 통해 상세하게 확인하실 수 있습니다.<br/>▶ 후기 관련 FAQ : https://www.musinsa.com/app/cs/faq?idx=40 <br/><br/>회원님의 편안한 쇼핑을 위해

In [ ]:
h ='https://image.msscdn.net/images/goods_img/20220125/2326935/2326935_11_220.jpg'
h.split('/')

['https:',
 '',
 'image.msscdn.net',
 'images',
 'goods_img',
 '20220125',
 '2326935',
 '2326935_11_220.jpg']

In [ ]:
[1,2,3,4,5][:2]

[1, 2]